In [1]:
import torch
import numpy as np
import cv2
print(torch.cuda.is_available())
from segment_anything import sam_model_registry, SamPredictor

True


In [56]:
#crop face
# Load image
image = cv2.imread("./assets/jonas.jpg")
output_width, output_height = 400, 500
corner_radius = 80  # pixels

# Load image
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

if len(faces) == 0:
    raise Exception("No face detected!")

# Choose the first face
(x, y, w, h) = faces[0]

# Define crop region centered around face with desired aspect ratio
center_x, center_y = x + w // 2, y + h // 2
desired_ratio = output_width / output_height

# Compute crop size keeping aspect ratio
if w / h > desired_ratio:
    crop_h = h
    crop_w = int(h * desired_ratio)
else:
    crop_w = w
    crop_h = int(w / desired_ratio)

# Expand the crop slightly for better framing
scale = 1.6
crop_w = int(crop_w * scale)
crop_h = int(crop_h * scale)

# Get top-left corner of crop
x1 = max(0, center_x - crop_w // 2)
y1 = max(0, center_y - crop_h // 2)
x2 = min(image.shape[1], x1 + crop_w)
y2 = min(image.shape[0], y1 + crop_h)

# Crop and resize
cropped = image[y1:y2, x1:x2]
resized = cv2.resize(cropped, (output_width, output_height))

# Create rounded corner mask
mask = np.zeros((output_height, output_width), dtype=np.uint8)
cv2.rectangle(mask, (0, 0), (output_width, output_height), 255, -1)

# Rounded rectangle via Gaussian blur
ksize = corner_radius if corner_radius % 2 == 1 else corner_radius + 1
mask = cv2.GaussianBlur(mask, (ksize, ksize), 0)

# Merge with alpha channel
rounded = cv2.cvtColor(resized, cv2.COLOR_BGR2BGRA)
rounded[:, :, 3] = mask

# Save result
cv2.imwrite("results/rounded_face.png", rounded)

True

In [70]:
def mouse_callback(event, x, y, flags, param):
    # if event == cv2.EVENT_MOUSEMOVE:
    #     print(f"Mouse moved to: ({x}, {y})")
    if event == cv2.EVENT_LBUTTONDOWN:
        print(f"Left button clicked at: ({x}, {y})")

# Load and show image
image = cv2.imread("assets/generated.png")
image = cv2.resize(image, (1600, 1200))
cv2.namedWindow("Image")
cv2.setMouseCallback("Image", mouse_callback)

while True:
    cv2.imshow("Image", image)
    if cv2.waitKey(1) & 0xFF == 27:  # ESC key to exit
        break

cv2.destroyAllWindows()

Left button clicked at: (153, 158)
Left button clicked at: (635, 704)


KeyboardInterrupt: 

In [10]:
# merge images
# Load background image
background = cv2.imread("assets/generated.png", cv2.IMREAD_COLOR)
background = cv2.resize(background, (1600, 1200))
# Load foreground (headshot) image with alpha channel
foreground = cv2.imread("results/rounded_face.png", cv2.IMREAD_UNCHANGED)  # BGRA

# Resize foreground to fit specified area
x1, y1 = 153, 158
x2, y2 = 635, 704
target_width = x2 - x1
target_height = y2 - y1
foreground = cv2.resize(foreground, (target_width, target_height))

# Separate channels
fg_rgb = foreground[:, :, :3]
fg_alpha = foreground[:, :, 3] / 255.0
fg_alpha = fg_alpha[..., np.newaxis]  # shape (h, w, 1)

# Get ROI from background
roi = background[y1:y2, x1:x2]

# Blend foreground and ROI
blended = (fg_rgb * fg_alpha + roi * (1 - fg_alpha)).astype(np.uint8)

# Put blended region back to background
background[y1:y2, x1:x2] = blended
font = cv2.FONT_HERSHEY_SIMPLEX
color = (255, 255, 255)  # White color
cv2.putText(background, "CEO Jonas", (100, 850), font, 5, color, 20, cv2.LINE_AA)
# Save or show
cv2.imwrite("results/final_composite.jpg", background)

True

In [5]:
# Load image
image = cv2.imread("./assets/Vehicle_img2.jpg")
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Load SAM model
sam = sam_model_registry["vit_b"](checkpoint="models/sam_vit_b_01ec64.pth")
sam.to("cuda")  # or "cpu" if you don't have a GPU
predictor = SamPredictor(sam)
predictor.set_image(image_rgb)

# User-provided foreground point (example: adjust as needed to a point on the car)
input_point = np.array([[image.shape[1]//2, image.shape[0]//2]])  # Approx center of image
input_label = np.array([1])  # 1 = foreground

# Predict mask
masks, scores, logits = predictor.predict(
    point_coords=input_point,
    point_labels=input_label,
    multimask_output=True,
)
# Choose the best mask based on score
best_idx = np.argmax(scores)
mask = masks[best_idx].astype(np.uint8)
# Get the predicted mask (foreground = car)
# mask = masks[0].astype(np.uint8)

# Convert image to BGRA
rgba = cv2.cvtColor(image, cv2.COLOR_BGR2BGRA)

# Set alpha channel: transparent for background (0), opaque for car (1)
rgba[:, :, 3] = mask * 255  # Keep car, make rest transparent

# Save the output image
cv2.imwrite("results/sam_car_only.png", rgba)

True

In [36]:
# merge result images

fg = cv2.imread("results/sam_background_only.png", cv2.IMREAD_UNCHANGED)  # includes alpha


# Load the background image
bg = cv2.imread("assets/background.jpg")

# Resize the foreground maintaining aspect ratio
h, w = fg.shape[:2]
aspect_ratio = w / h
bg_h, bg_w = bg.shape[:2]

# Calculate new dimensions for foreground
if aspect_ratio > bg_w / bg_h:
    new_w = bg_w
    new_h = int(bg_w / aspect_ratio)
else:
    new_h = bg_h
    new_w = int(bg_h * aspect_ratio)

fg_resized = cv2.resize(fg, (new_w, new_h))

# Create a new blank canvas with the background size
pad_top = (bg_h - new_h) // 2
pad_left = (bg_w - new_w) // 2

# Create a padded foreground
padded_fg = np.zeros((bg_h, bg_w, 4), dtype=np.uint8)
padded_fg[pad_top:pad_top+new_h, pad_left:pad_left+new_w] = fg_resized

# Split the padded foreground
b, g, r, a = cv2.split(padded_fg)

# Normalize the alpha mask
alpha = a.astype(float) / 255
alpha = np.stack([alpha] * 3, axis=-1)

# Perform alpha blending: Merge foreground with background
fg_rgb = cv2.merge([b, g, r])
result = (fg_rgb * alpha + bg * (1 - alpha)).astype(np.uint8)

# Save the final merged result
cv2.imwrite("results/merged_result_with_padding.jpg", result)

True

In [37]:
text = "CEO Jonas"
height, width, _ = result.shape

# Dynamically set font scale based on image height
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = height / 600  # Tune this ratio for your preferred size
thickness = int(height / 300)  # Adjust thickness relative to image height
color = (255, 255, 255)  # White color

# Get text size
text_size, _ = cv2.getTextSize(text, font, font_scale, thickness)
text_width, text_height = text_size

# Calculate centered position
x = (width - text_width) // 2
y =  height - text_height

# Optional: Add dark rectangle background behind text
rect_padding = 10
cv2.rectangle(
result,
(x - rect_padding, y - text_height - rect_padding),
(x + text_width + rect_padding, y + rect_padding),
(0, 255, 0),  # Black background
cv2.FILLED
)

# Put text on image
cv2.putText(result, text, (x, y), font, font_scale, color, thickness, cv2.LINE_AA)

# Save the result
cv2.imwrite("results/image_with_text.png", result)

True